# Installing necessary dependencies

In [10]:

%pip install torch torchvision scikit-learn opencv-python matplotlib


# Importing Libraries

In [11]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix



# Transform for preprocessing

In [27]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize between -1 and 1
])

# Download and load dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Check data shapes
print(f"Train data: {len(train_loader.dataset)} samples")
print(f"Test data: {len(test_loader.dataset)} samples")


Train data: 60000 samples
Test data: 10000 samples


# Defining the Neural Network

In [13]:
class DigitClassifier(nn.Module):
    def __init__(self):
        super(DigitClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)  # 10 classes for digits 0-9
        )
    
    def forward(self, x):
        return self.model(x)

# Initialize the model
model = DigitClassifier()
print(model)


DigitClassifier(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=10, bias=True)
  )
)


# Define Loss and Optimizer

In [14]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training the Model

In [15]:
# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")


Epoch 1/5, Loss: 0.3883
Epoch 2/5, Loss: 0.1838
Epoch 3/5, Loss: 0.1366
Epoch 4/5, Loss: 0.1092
Epoch 5/5, Loss: 0.0927


# Evaluate the Model

In [16]:
# Evaluation
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.numpy())

# Classification report
print(classification_report(y_true, y_pred, digits=4))


              precision    recall  f1-score   support

           0     0.9697    0.9786    0.9741       980
           1     0.9843    0.9912    0.9877      1135
           2     0.9898    0.9448    0.9668      1032
           3     0.9675    0.9733    0.9704      1010
           4     0.9618    0.9756    0.9687       982
           5     0.9738    0.9596    0.9667       892
           6     0.9532    0.9770    0.9649       958
           7     0.9429    0.9796    0.9609      1028
           8     0.9739    0.9589    0.9664       974
           9     0.9695    0.9445    0.9568      1009

    accuracy                         0.9686     10000
   macro avg     0.9686    0.9683    0.9683     10000
weighted avg     0.9689    0.9686    0.9686     10000



# Test with Custom Handwritten Digits (Using OpenCV)

In [29]:


# Create a blank white canvas
canvas = np.ones((280, 280), dtype=np.uint8) * 255

# Function to handle mouse events for drawing
def draw(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN or (flags == cv2.EVENT_LBUTTONDOWN and event == cv2.EVENT_MOUSEMOVE):
        cv2.circle(canvas, (x, y), 10, (0, 0, 0), -1)  # Draw black circle (brush) on mouse press or move

# Setup OpenCV window and set callback for mouse event
cv2.namedWindow('Draw a digit')
cv2.setMouseCallback('Draw a digit', draw)

# Draw on canvas until 'Enter' is pressed
while True:
    cv2.imshow('Draw a digit', canvas)
    if cv2.waitKey(1) & 0xFF == 13:  # Press Enter to stop drawing
        break

cv2.destroyAllWindows()

# Preprocess the drawn digit
digit = cv2.resize(canvas, (28, 28))  # Resize to MNIST size

# Add padding to center the digit if needed
top, bottom, left, right = [10, 10, 10, 10]  # Adjust padding if needed
digit = cv2.copyMakeBorder(digit, top, bottom, left, right, cv2.BORDER_CONSTANT, value=255)  # Add padding
digit = cv2.resize(digit, (28, 28))  # Resize back to 28x28

digit = digit / 255.0  # Normalize to [0, 1] range
digit = torch.tensor(digit, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions

# Prediction using trained model
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    output = model(digit)
    _, prediction = torch.max(output, 1)  # Get predicted class
print(f"Predicted Digit: {prediction.item()}")


Predicted Digit: 3
